<a href="https://colab.research.google.com/github/jeffblackadar/sensation/blob/main/sensory_map_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Package installation, Mount Google Drive

In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Install packages
!apt-get install libgeos-dev
!pip install https://github.com/matplotlib/basemap/archive/master.zip
!pip install geopandas
!pip install contextily

# 2. Relocate map coordinates to get ready to unroll map at your location's latitude longitude
* Clicking a location in Google Maps will provide this.

In [3]:
# Input the location to unroll the map at.
local_lat = 45.475871
local_lon = -75.549315

#local_lat = 45.477661
#local_lon = -75.536143

#local_lat = 43.586368
#local_lon = -79.542910

#local_lat = 43.590619
#local_lon = -79.539579

In [4]:
shapefiles_path = "/content/drive/MyDrive/github/sensation/shapefiles/"
from geographiclib.geodesic import Geodesic
import geopy
from geopy.distance import VincentyDistance
import geopandas as gpd
from shapely.geometry import Point, Polygon
import os

def transition_point(origin_lat, origin_lon, dest_lat, dest_lon, new_map_origin_lat, new_map_origin_lon):
    #print(origin_lat, origin_lon, dest_lat, dest_lon, new_map_origin_lat, new_map_origin_lon)  
    geod = Geodesic.WGS84.Inverse(origin_lat, origin_lon, dest_lat, dest_lon)
    distance = geod['s12']
    bearing = geod['azi1']
    origin = geopy.Point(new_map_origin_lat, new_map_origin_lon)
    destination = VincentyDistance(meters=distance).destination(origin, bearing)
    lat2, lon2 = destination.latitude, destination.longitude
    return(lat2, lon2)


geotif_crs = 4326
new_map_shp = gpd.GeoDataFrame()
new_map_shp['geometry'] = None
new_map_shp.crs = ("EPSG:" + str(geotif_crs))
new_map_shp.geometry = new_map_shp.geometry.to_crs(crs=geotif_crs)
new_map_shp.to_crs(crs=geotif_crs)
new_map_shp = new_map_shp.to_crs(epsg=geotif_crs)

print("#--------------------------------------------------------")
print("# Copy this below, paste in the cell in section 3 of this notebook to provide instructions to generate the new map")
print("#--------------------------------------------------------")

h_df = gpd.read_file(os.path.join(shapefiles_path,"hopewell_selected_elements.shp"))
for index, row in h_df.iterrows():
    #print(row['type'], row['name'], row['geometry'])
    p = row['geometry']
    lat2, lon2 = transition_point(40.20595294149071, -75.77220799054444, p.y, p.x, local_lat, local_lon)

    point = Point(lon2, lat2)
    new_p_row = {'id':row['id'], 'geometry':point, 'type':row['type'], 'name':row['name']}
    t_name = str(row['name'])
    if(t_name.lower()=='none'):
        t_name=""
    t_desc = str(row['desc'])
    if(t_desc.lower()=='none'):
        t_desc=""
    print('gm.genMarker('+str(lat2)+', '+str(lon2)+',"'+str(row['type'])+'","'+t_name+'","'+t_desc+'")')
    new_map_shp = new_map_shp.append(new_p_row, ignore_index=True)

new_map_shp.to_file(os.path.join(shapefiles_path,"new_map.shp"))


new_map_shp_poly = gpd.GeoDataFrame()
new_map_shp_poly['geometry'] = None
new_map_shp_poly.crs = ("EPSG:" + str(geotif_crs))
new_map_shp_poly.geometry = new_map_shp_poly.geometry.to_crs(crs=geotif_crs)
new_map_shp_poly.to_crs(crs=geotif_crs)
new_map_shp_poly = new_map_shp_poly.to_crs(epsg=geotif_crs)
polygon_points_array = []

h_df = gpd.read_file(os.path.join(shapefiles_path,"hopewell_selected_elements_polys.shp"))
for index, row in h_df.iterrows():
    poly = row['geometry']
    polygon_points = ""
    comma = ""
    for p in poly.exterior.coords:
        lat2, lon2 = transition_point(40.20595294149071, -75.77220799054444, p[1], p[0], local_lat, local_lon)
        polygon_points = polygon_points + comma + "["+str(lat2)+","+str(lon2)+"]"
        comma = ","
        #print(lat2, lon2)
    #/content/drive/MyDrive/crane_obs/crane_hopewell/hopewell_selected_elements.shp
        point = Point(lon2, lat2)
        polygon_points_array.append(point)
    polygon = Polygon(polygon_points_array)
    new_p_row = {'id':row['id'], 'geometry':polygon, 'type':row['type'], 'name':row['name']}
    print("gm.genPolygon('["+polygon_points+"]','"+str(row['type'])+"')")
    new_map_shp_poly = new_map_shp_poly.append(new_p_row, ignore_index=True)

new_map_shp_poly.to_file(os.path.join(shapefiles_path,"new_map_polys.shp"))

#test marker
origin = geopy.Point(local_lat, local_lon)
destination = VincentyDistance(meters=2000).destination(origin, 270)
lat2, lon2 = destination.latitude, destination.longitude
print('gm.genTestMarker('+str(lat2)+', '+str(lon2)+')')
print("")
print("#--------------------------------------------------------")
print("# Copy this above, paste in the cell in section 3 of this notebook to provide instructions to generate the new map")
print("#--------------------------------------------------------")

#--------------------------------------------------------
# Copy this below, paste in the cell in section 3 of this notebook to provide instructions to generate the new map
#--------------------------------------------------------
gm.genMarker(45.47890524246566, -75.56611403321713,"RCH","RCH","")
gm.genMarker(45.478045775348, -75.56241738538371,"RCH","","")
gm.genMarker(45.479084619032285, -75.5606251644505,"RCH","","")
gm.genMarker(45.479801606521505, -75.55691209024718,"RCH","","")
gm.genMarker(45.478952924359234, -75.55542219985323,"RCH","","")
gm.genMarker(45.4793838133057, -75.55354605510716,"RCH","","")
gm.genMarker(45.479780549057914, -75.5525676092125,"RCH","","")
gm.genMarker(45.470290416814905, -75.55228904428883,"RCH","","")
gm.genMarker(45.47033550770272, -75.55056360470792,"RCH","","")
gm.genMarker(45.46974425824116, -75.55151372730964,"RCH","","")
gm.genMarker(45.468732503626555, -75.55196028059133,"RCH","","")
gm.genMarker(45.4692836321735, -75.55304573662096,"RCH","",""

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import geopandas as gpd
import pandas as pd
import contextily as ctx

print("Original Map")
h_df = gpd.read_file(os.path.join(shapefiles_path,"hopewell_selected_elements.shp"))

#print(h_df.crs)
# Change crs to one compatible with basemap
h_df = h_df.to_crs(epsg=3857) #3857

ax = h_df.plot(figsize=(20, 20), alpha=0.5, edgecolor='k')
ctx.add_basemap(ax, zoom=15)

h_df['coords'] = h_df['geometry'].apply(lambda x: x.representative_point().coords[:])
h_df['coords'] = [coords[0] for coords in h_df['coords']]
for idx, row in h_df.iterrows():
    plt.annotate(s=row['type'], xy=row['coords'], horizontalalignment='center')

print("Unrolled map at your location")
new_map_df = gpd.read_file(os.path.join(shapefiles_path,"new_map.shp"))

print(new_map_df.crs)
# Change crs to one compatible with basemap
new_map_df = new_map_df.to_crs(epsg=3857) #3857
print(new_map_df.crs)

ax = new_map_df.plot(figsize=(20, 20), alpha=0.5, edgecolor='k')
ctx.add_basemap(ax, zoom=13)

new_map_df['coords'] = new_map_df['geometry'].apply(lambda x: x.representative_point().coords[:])
new_map_df['coords'] = [coords[0] for coords in new_map_df['coords']]
for idx, row in new_map_df.iterrows():
    plt.annotate(s=row['type'], xy=row['coords'], horizontalalignment='center')

new_map_df = gpd.read_file(os.path.join(shapefiles_path,"new_map_polys.shp"))
#new_map_df.boundary.plot(ax=ax,color="red",alpha=0.3)

# 3. Generate the map to display at your location

In [11]:
import json
class GenerateLeafletMap:
    def __init__(self,html_file_name):
        self.html_file_name = html_file_name
        self.html_file = open(self.html_file_name, 'w') 
        self.markers = 0
        self.polygons = 0
        self.polylines = 0
        self.layers = 0
        self.type_data = {}
    def close(self):
        self.html_file.close()

    def set_type_data_dict(self,type_data_dict):
        self.type_data = type_data_dict

    def genMapHTMLTop(self):
        self.html_file.write('<!DOCTYPE html>\n')
        self.html_file.write('<html>\n')
        self.html_file.write('<head><meta http-equiv="Content-Type" content="text/html; charset=utf-8">\n')
        self.html_file.write('<title>map title</title>\n')
        self.html_file.write('<link href="https://cdnjs.cloudflare.com/ajax/libs/leaflet/0.7.5/leaflet.css" rel="stylesheet" />\n')
        self.html_file.write('<script src="https://cdnjs.cloudflare.com/ajax/libs/leaflet/0.7.5/leaflet.js"></script>\n')
        self.html_file.write('<script src="https://code.jquery.com/jquery-2.1.4.min.js"></script>\n')
        self.html_file.write('<style type="text/css">#my-map {\n')
        self.html_file.write('  width:960px;\n')
        self.html_file.write('  height:700px;\n')
        self.html_file.write('}\n')
        self.html_file.write('h1 {\n')
        self.html_file.write('  color: #AA0000;\n')
        self.html_file.write('  font-size: 24px;\n')
        self.html_file.write('  font-family: verdana;\n')
        self.html_file.write('}\n')
        self.html_file.write('h2 {\n')
        self.html_file.write('  color: #000000;\n')
        self.html_file.write('  font-size: 20px;\n')
        self.html_file.write('  font-family: verdana;\n')
        self.html_file.write('}\n')
        self.html_file.write('h3 {\n')
        self.html_file.write('  color: #000000;\n')
        self.html_file.write('  font-size: 16px;\n')
        self.html_file.write('  font-family: verdana;\n')
        self.html_file.write('}\n')    
        self.html_file.write('p {\n')
        self.html_file.write('  color: #996600;\n')
        self.html_file.write('  font-family: verdana;\n')
        self.html_file.write('}\n')
        self.html_file.write('li {\n')
        self.html_file.write('  color: #996600;\n')
        self.html_file.write('  font-family: verdana;\n')
        self.html_file.write('}\n')  
        self.html_file.write('th {\n')
        self.html_file.write('  text-align: right;\n')
        self.html_file.write('  font-family: verdana;\n')
        self.html_file.write('  color: #663300;\n')
        self.html_file.write('}\n')
        self.html_file.write('td {\n')
        self.html_file.write('  text-align: right;\n')
        self.html_file.write('  font-family: verdana;\n')
        self.html_file.write('  color: #663300;\n')
        self.html_file.write('}\n')
        self.html_file.write('</style>\n')
        self.html_file.write('</head>\n')
        self.html_file.write('<body>\n')

    def genMapHTMLH1(self,map_title):
        self.html_file.write('<h1>'+map_title+'</h1>\n')
        self.html_file.write('<div id="description">description</div>\n')
        self.html_file.write('<div id="latitude">latitude</div>\n')
        self.html_file.write('<div id="longitude">longitude</div>\n')
        self.html_file.write('<div id="description_point"></div>\n')
        self.html_file.write('<div id="description_poly"></div>\n')
                    
    def genMapHTMLScriptTop(self): 
        self.html_file.write('  <div id="my-map"></div>\n')
        self.html_file.write('    <script>\n')
        self.html_file.write('    // This file was generated by this R program:  https://github.com/jeffblackadar/oc-transpo-maps/blob/master/oc_4_gen_html.r\n')
        self.html_file.write('var map;\n')
        self.html_file.write("map = L.map('my-map');\n")
        self.html_file.write('var markers = [];\n')
        self.html_file.write('var polygons = [];\n')
        self.html_file.write('var markers_data = [];\n')
        self.html_file.write('var polygons_data = [];\n')

        self.html_file.write("// Thank you:\n")
        self.html_file.write("// https://stackoverflow.com/questions/27928/calculate-distance-between-two-latitude-longitude-points-haversine-formula\n")
        self.html_file.write("function distance(lat1, lon1, lat2, lon2) {\n")
        self.html_file.write("  var p = 0.017453292519943295;    // Math.PI / 180\n")  
        self.html_file.write("  var c = Math.cos;\n")
        self.html_file.write("  var a = 0.5 - c((lat2 - lat1) * p)/2 + \n")
        self.html_file.write("          c(lat1 * p) * c(lat2 * p) * \n")
        self.html_file.write("          (1 - c((lon2 - lon1) * p))/2;\n")
        self.html_file.write("  return 12742 * Math.asin(Math.sqrt(a)); // 2 * R; R = 6371 km\n")
        self.html_file.write("}\n")

        self.html_file.write("//https://stackoverflow.com/questions/31790344/determine-if-a-point-reside-inside-a-leaflet-polygon\n")
        self.html_file.write("function isMarkerInsidePolygon(marker, poly) {\n")
        self.html_file.write("    var polyPoints = poly.getLatLngs();       \n")
        self.html_file.write("    var x = marker.getLatLng().lat, y = marker.getLatLng().lng;\n")
        self.html_file.write("    var inside = false;\n")
        self.html_file.write("    for (var i = 0, j = polyPoints.length - 1; i < polyPoints.length; j = i++) {\n")
        self.html_file.write("        var xi = polyPoints[i].lat, yi = polyPoints[i].lng;\n")
        self.html_file.write("        var xj = polyPoints[j].lat, yj = polyPoints[j].lng;\n")
        self.html_file.write("        var intersect = ((yi > y) != (yj > y)) && (x < (xj - xi) * (y - yi) / (yj - yi) + xi);\n")
        self.html_file.write("        if (intersect) inside = !inside;\n")
        self.html_file.write("    }\n")
        self.html_file.write("    return inside;\n")
        self.html_file.write("};\n")
        self.html_file.write("function update_per_proximity(marker) {\n")
        self.html_file.write("    var marker_lat = marker.getLatLng().lat, marker_lon = marker.getLatLng().lng;\n")

        self.html_file.write("// Polygons - check if marker is inside.  If so, light it up.\n")  
        self.html_file.write("    for (cn=0;cn<polygons.length;cn++){\n")
        self.html_file.write("        if(isMarkerInsidePolygon(marker, polygons[cn])==true){\n")
        self.html_file.write("            polygons[cn].setStyle({fillOpacity: 0.7, weight:2,opacity: 0.3});\n")
        self.html_file.write('            var popup_text_poly = polygons_data[cn].sight + " " + polygons_data[cn].hearing + " " + polygons_data[cn].feeling + " " + polygons_data[cn].smell;\n')        
        self.html_file.write("            document.getElementById('description_poly').innerHTML = popup_text_poly;\n")
        self.html_file.write("            var audio_temp = polygons_data[cn].sound;\n")
        self.html_file.write("            if(audio_temp.trim().length>0){audio_temp = '<audio controls><source src=\"sounds/'+audio_temp+'\"> Your browser does not support the audio element. </audio> <i>Sounds are fictional representations.</i>'}")
        self.html_file.write("            polygons[cn]._popup.setContent(popup_text_poly + audio_temp + ' <i>Landscape types and descriptions are imagined based on the map.</i>')\n")

        self.html_file.write("        }\n")
        self.html_file.write("    }\n")
#    "hill":'{"color":"brown","sight":"hilly terrain","hearing":"wind blowing through trees","smell":"damp leaves","feeling":"cool, damp"}',
        self.html_file.write("    for (cn=0;cn<markers.length;cn++){\n")
        self.html_file.write("        dist_to = distance(marker_lat, marker_lon,markers[cn].getLatLng().lat,markers[cn].getLatLng().lng)\n")
        self.html_file.write("        // adjust distance for proximity sensitivity\n")
        self.html_file.write("        if(dist_to < 0.1){\n")

        self.html_file.write("            if(markers_data[cn].type=='rch'){\n")
        self.html_file.write("                markers[cn].setIcon(iconRCH);\n")
        self.html_file.write("            }\n")
        self.html_file.write("\n")
        self.html_file.write("            if(markers_data[cn].type=='furnace'){\n")
        self.html_file.write("                markers[cn].setIcon(iconFurnace);\n")
        self.html_file.write("            }\n")
        self.html_file.write("\n")
        self.html_file.write("            if(markers_data[cn].type=='business'){\n")
        self.html_file.write("                markers[cn].setIcon(iconBusiness);\n")
        self.html_file.write("            }\n")
        self.html_file.write("\n")
        self.html_file.write("            if(markers_data[cn].type=='house'){\n")
        self.html_file.write("                markers[cn].setIcon(iconHouse);\n")
        self.html_file.write("            }\n")
        self.html_file.write("\n")    
        self.html_file.write('            var desc_temp = markers_data[cn].desc;')
        self.html_file.write('            if(desc_temp.trim().length>0){desc_temp = "This is described on the map as: "+desc_temp+". "};')
        self.html_file.write('            var popup_text = markers_data[cn].sight + " " + markers_data[cn].hearing + " " + markers_data[cn].feeling + " " + markers_data[cn].smell + " " + desc_temp;\n')
        self.html_file.write("            document.getElementById('description_point').innerHTML = popup_text;\n")
        self.html_file.write("            var audio_temp = markers_data[cn].sound;\n")
        self.html_file.write("            if(audio_temp.trim().length>0){audio_temp = '<audio controls><source src=\"sounds/'+audio_temp+'\"> Your browser does not support the audio element. </audio> <i>Sounds are fictional representations.</i>'}")
        self.html_file.write("            markers[cn]._popup.setContent(popup_text + audio_temp)\n")


        self.html_file.write("        } // if dist_to\n")
        self.html_file.write("    } // for\n")

        self.html_file.write("} // function\n")                      
      
        self.html_file.write('var iconHouse = L.icon({\n')
        self.html_file.write('    iconUrl: "icons/house.png",\n')
        self.html_file.write('    //shadowUrl: "leaf-shadow.png",\n')

        self.html_file.write('    iconSize:     [30, 37], // size of the icon\n')
        self.html_file.write('    //shadowSize:   [50, 64], // size of the shadow\n')
        self.html_file.write('    iconAnchor:   [15, 36], // point of the icon which will correspond to markers location\n')
        self.html_file.write('    //shadowAnchor: [4, 62],  // the same for the shadow\n')
        self.html_file.write('    popupAnchor:  [-3, -38] // point from which the popup should open relative to the iconAnchor\n')
        self.html_file.write('});\n')

        self.html_file.write('var iconRCH = L.icon({\n')
        self.html_file.write('    iconUrl: "icons/rch.png",\n')
        self.html_file.write('    iconSize:     [35, 33], // size of the icon\n')
        self.html_file.write('    iconAnchor:   [17, 32], // point of the icon which will correspond to \n')
        self.html_file.write('    popupAnchor:  [-3, -34] // point from which the popup should open relative to the iconAnchor\n')
        self.html_file.write('});\n')
      
        self.html_file.write('var iconFurnace = L.icon({\n')
        self.html_file.write('    iconUrl: "icons/furnace.png",\n')
        self.html_file.write('    iconSize:     [34, 38], // size of the icon\n')
        self.html_file.write('    iconAnchor:   [17, 37], // point of the icon which will correspond to \n')
        self.html_file.write('    popupAnchor:  [-3, -39] // point from which the popup should open relative to the iconAnchor\n')
        self.html_file.write('});\n')

        self.html_file.write('\n')
        self.html_file.write('var iconBusiness = L.icon({\n')
        self.html_file.write('    iconUrl: "icons/business.png",\n')
        self.html_file.write('    iconSize:     [32, 32],\n')
        self.html_file.write('    iconAnchor:   [16, 32],\n')
        self.html_file.write('    popupAnchor:  [-3, -33]\n')
        self.html_file.write('});\n')

        self.html_file.write('\n')
        self.html_file.write('var iconTravel = L.icon({\n')
        self.html_file.write('    iconUrl: "icons/travel.png",\n')
        self.html_file.write('    iconSize:     [32, 32],\n')
        self.html_file.write('    iconAnchor:   [16, 32],\n')
        self.html_file.write('    popupAnchor:  [-3, -33]\n')
        self.html_file.write('});\n')

        self.html_file.write('\n')
        self.html_file.write('var iconComputer = L.icon({\n')
        self.html_file.write('    iconUrl: "icons/computer.png",\n')
        self.html_file.write('    iconSize:     [32, 32],\n')
        self.html_file.write('    iconAnchor:   [16, 32],\n')
        self.html_file.write('    popupAnchor:  [-3, -33]\n')
        self.html_file.write('});\n')

        self.html_file.write('\n')
        self.html_file.write('var iconQuestion = L.icon({\n')
        self.html_file.write('    iconUrl: "icons/question.png",\n')
        self.html_file.write('    iconSize:     [32, 32],\n')
        self.html_file.write('    iconAnchor:   [16, 32],\n')
        self.html_file.write('    popupAnchor:  [-3, -33]\n')
        self.html_file.write('});\n')
                    
    def genMapHTMLMapTop(self):
        self.html_file.write("\n")
                      
        self.html_file.write("      //basemap.fitBounds(geojson.getBounds());\n")
        self.html_file.write("      map.setView([45.468, -75.553], 15);\n")
        self.html_file.write("      basemap.addTo(map);\n")
        self.html_file.write("\n")
        self.html_file.write("      // add legend control layers - global variable with (null, null) allows indiv basemaps and overlays to be added inside functions below\n")
        self.html_file.write("      var controlLayers = L.control.layers(null, null, {\n")
        self.html_file.write('        position: "topleft",\n')
        self.html_file.write("        collapsed: false // false = open by default\n")
        self.html_file.write("      }).addTo(map);\n")
        self.html_file.write("\n")
        #writeLines("      var markers = [];", self$outputFileHtmlCon)
                      

    def genHTMLBaseMap(self):
        self.html_file.write('  window.onload = function() {\n')
        self.html_file.write("    var basemap = L.tileLayer('https://{s}.tile.osm.org/{z}/{x}/{y}.png', {\n")
        self.html_file.write("      attribution: '&copy; "+'<a href="https://osm.org/copyright">OpenStreetMap</a>'+"'\n")
        self.html_file.write("    });\n")

    def genMarker(self,marker_lat, marker_long, marker_type, marker_name, marker_desc):
        type_data_row = self.type_data[marker_type.lower()]
        #add more data to the json for the marker
        type_data_row = type_data_row[:-1]+',"name":"'+marker_name+'","desc":"'+marker_desc+'"}'
        type_data_row_json = json.loads(type_data_row)  
        self.html_file.write('  markers.push(L.marker(['+str(marker_lat)+','+str(marker_long)+'], {icon: iconQuestion}).addTo(map));\n')
        self.html_file.write('     markers['+str(self.markers)+"].bindPopup('Approach the landmark to see what it is.').openPopup();\n")
        self.html_file.write('     markers_data.push('+type_data_row+')\n')
        self.markers = self.markers +1


    def genPolygon(self,polygon_coods_json, polygon_type):
        # Thanks to: https://gis.stackexchange.com/questions/290609/entering-manual-coordinates-to-draw-polygon-in-leaflet
        fill_color="black"
        type_data_row = self.type_data[polygon_type.lower()]
        type_data_row_json = json.loads(type_data_row)

        fill_color = type_data_row_json['color']
        
        self.html_file.write('var coords = '+polygon_coods_json+';\n')
                      #writeLines(paste0('var a = JSON.parse(coords);\n')
        self.html_file.write('polygons.push(L.polygon(coords, {\n')
        self.html_file.write('  color: "white",\n')
        self.html_file.write('  fillColor: "'+fill_color+'",\n')
        self.html_file.write('  weight: 0,\n')
        self.html_file.write('  opacity: 0,\n')
        self.html_file.write('  fillOpacity: 0\n')
        self.html_file.write('}).addTo(map));\n')
        self.html_file.write('     polygons['+str(self.polygons)+"].bindPopup('Approach the landmark to see what it is.').openPopup();\n")
        self.html_file.write('     polygons_data.push('+type_data_row+')\n')
        self.html_file.write('\n')
        self.polygons = self.polygons +1


    def genPolyline(self,p1_lat, p1_long, p2_lat, p2_long):
        self.polylines = self.polylines +1
        self.html_file.write('  var pointA = new L.LatLng(',paste0(p1_lat),',',paste0(p1_long),' );\n')
        self.html_file.write('  var pointB = new L.LatLng(',paste0(p2_lat),',',paste0(p2_long),');\n')
        self.html_file.write('  var pointList = [pointA, pointB];\n')
        self.html_file.write('  \n')
        self.html_file.write('  var polyline'+str(self.polylines)+' = new L.Polyline(pointList, {\n')
        self.html_file.write("      color: 'red',\n")
        self.html_file.write('      weight: 3,\n')
        self.html_file.write('      opacity: 0.5,\n')
        self.html_file.write('      smoothFactor: 1\n')
        self.html_file.write('  });\n')
        self.html_file.write('  polyline'+str(self.polylines)+'.addTo(map);\n')

    def genTestMarker(self,marker_lat, marker_long,):
        self.html_file.write("// To simulate walking around this is a draggable marker. This will greatly reduce testing time.\n")
        self.html_file.write("// Thanks to\n")
        self.html_file.write("// https://stackoverflow.com/questions/57384822/leaflet-detect-when-marker-goes-into-and-out-of-a-circle\n")
        self.html_file.write("var test_marker = new L.marker(["+str(marker_lat)+","+str(marker_long)+"],{\n")
 
        self.html_file.write("    draggable: true,\n")
        self.html_file.write("    autoPan: true\n")
        self.html_file.write("}).addTo(map);\n")
        self.html_file.write("test_marker.setIcon(iconComputer)\n")

        self.html_file.write("test_marker.on('dragend', function (e) {\n")
        self.html_file.write("    document.getElementById('latitude').innerHTML = test_marker.getLatLng().lat;\n")
        self.html_file.write("    document.getElementById('longitude').innerHTML = test_marker.getLatLng().lng;\n")
        self.html_file.write("    document.getElementById('description').innerHTML = distance(test_marker.getLatLng().lat,test_marker.getLatLng().lng,markers[13].getLatLng().lat,markers[13].getLatLng().lng);\n")

        self.html_file.write("    update_per_proximity(test_marker)\n")      
                    

    def genHTMLEndMap(self):
        self.html_file.write("      controlLayers.addOverlay(basemap, 'OpenStreetMap');\n")
        self.html_file.write("      \n")
        self.html_file.write("   // });\n")
        self.html_file.write("});\n")
        self.html_file.write("  };\n")


        self.html_file.write("   // https://gis.stackexchange.com/questions/182068/getting-current-user-location-automatically-every-x-seconds-to-put-on-leaflet\n")
        self.html_file.write("var current_position, current_accuracy;\n")
        self.html_file.write("function onLocationFound(e) {\n")
        self.html_file.write("    // if position defined, then remove the existing position marker and accuracy circle from the map\n")
        self.html_file.write("    if (current_position) {\n")
        self.html_file.write("        map.removeLayer(current_position);\n")
        #writeLines(paste0("       map.removeLayer(current_accuracy);\n")
        self.html_file.write("    }\n")
        self.html_file.write("    var radius = e.accuracy / 2;\n")
        self.html_file.write("    current_position = L.marker(e.latlng).addTo(map)\n")
        self.html_file.write('        .bindPopup("You are within " + radius + " meters from this point");\n')
        self.html_file.write("    current_position.setIcon(iconTravel)\n")
        #writeLines(paste0("current_accuracy = L.circle(e.latlng, radius).addTo(map);\n")
        self.html_file.write("    update_per_proximity(current_position);\n")
        self.html_file.write("}\n")
        self.html_file.write("function onLocationError(e) {\n")
        self.html_file.write("    alert(e.message);\n")
        self.html_file.write("}\n")
        self.html_file.write("map.on('locationfound', onLocationFound);\n")
        self.html_file.write("map.on('locationerror', onLocationError);\n")
        self.html_file.write("// wrap map.locate in a function    \n")
        self.html_file.write("function locate() {\n")
        self.html_file.write("    map.locate({setView: true, maxZoom: 16});\n")
        self.html_file.write("}\n")

        self.html_file.write("// call locate every 3 seconds... forever\n")
        self.html_file.write("setInterval(locate, 3000);\n")

        self.html_file.write("</script>\n")
                    
    def genHTMLEndBody(self):
        self.html_file.write("<hr>\n")
        #self.html_file.write("<p>Page last modifed: ", format(Sys.Date(),"%d %B, %Y"),"</p>\n")
        self.html_file.write("</body>\n")
        self.html_file.write("</html>\n")

    def genLayer(self,geojsonfile):
        self.layers = self.layers +1
        #print(paste0("To map for ", railYear, ", adding: "))
        #print(output_dbRows_route_style[i_route_style, 1])
        layerName = "layer" + str(self.layers)
                      
        self.html_file.write("\n")
        self.html_file.write("// layer for "+geojsonfile+"\n")
        self.html_file.write("var ",layerName,";\n")
        self.html_file.write("      $.ajax({\n")
        self.html_file.write("        type: 'POST',\n")
        self.html_file.write("        url: '",geojsonfile,"',\n")
        self.html_file.write("        dataType: 'json',\n")
        self.html_file.write("        success: function(response) {\n")
        self.html_file.write("          ",layerName," = L.geoJson(response, {\n")
        self.html_file.write("             onEachFeature: function (feature, layer) {\n")
        self.html_file.write("                 layer.bindPopup('<p>'+feature.properties.RTE_TYPE+'</p><p>'+feature.properties.MODE+'</p>');\n")  
        self.html_file.write("               }\n")
        self.html_file.write("             });\n")
        self.html_file.write("          ",layerName,".setStyle({\n")
        self.html_file.write("            color: 'purple',\n")
        self.html_file.write("            weight: 2,\n")
        self.html_file.write('            opacity: 1\n')
        self.html_file.write("          });\n")
        self.html_file.write("          ",layerName,".addTo(map);\n")
        self.html_file.write("          controlLayers.addOverlay(",layerName,", '<span style=",'"color: purple"',">route</span>');\n")
        self.html_file.write("        }\n")
        self.html_file.write("      });\n")
        self.html_file.write("\n")
                      

gm = GenerateLeafletMap("/content/drive/MyDrive/github/sensation/userloc.html")
gm.genMapHTMLTop()


type_data_dict={
    "hill":'{"type":"hill","color":"brown","sight":"This is treed hilly terrain. It feels cooler here. You hear the wind in the trees and smell the forest.","hearing":"","smell":"","feeling":"","sound":""}',
    "rch":'{"type":"rch","color":"","sight":"This is a flat circle ~10m in diameter. It has a residue of charcoal and wood ash. The date of construction of this charcoal hearth is unknown. It is possible it was constructed later than 1860.","hearing":"wind blowing through trees","smell":"ash","feeling":"cool, damp","sound":"rch.wav"}',
    "forest":'{"type":"forest","color":"green","sight":"This is treed, fairly level terrain.  It feels cooler here. You hear the wind in the trees and smell the forest.","hearing":"","smell":"","feeling":"","sound":""}',
    "field":'{"type":"field","color":"wheat","sight":"This is a fairly level open field with some gardens near houses. Some cows are grazing and you pick up a faint scent of manure.","hearing":"","smell":"","feeling":"","sound":""}',
    "road":'{"type":"road","color":"gray","sight":"This is a road with a surface of stones and slag from the furnace.","hearing":"","smell":"","feeling":"","sound":"road.wav"}',
    "path":'{"type":"path","color":"gray","sight":"This is a muddy path and you hear your footsteps as you travel it.","hearing":"","smell":"","feeling":"","sound":"path.wav"}',
    "creek":'{"type":"creek","color":"blue","sight":"This is a creek and you hear the sound of water moving over rocks. The smell of fresh water is pleasant.","hearing":"","smell":"","feeling":"","sound":"creek.wav"}',
    "business":'{"type":"business","color":"","sight":"This is a commerical building. A wagon and horses are outside.","hearing":"","smell":"","feeling":"","sound":""}',
    "furnace":'{"type":"furnace","color":"","sight":"The furnace is a large building and you smell the smoke of burning coal and hear the roar of the fire.","hearing":"","smell":"","feeling":"","sound":"furnace.wav"}',
    "house":'{"type":"This is a house. You can smell wood smoke from a chimney and hear people talking.","color":"","sight":"a house","hearing":"","smell":"","feeling":"","sound":""}'
}
gm.set_type_data_dict(type_data_dict)

gm.genMapHTMLH1("Hopewell, Pennsylvania")
gm.genMapHTMLScriptTop()
gm.genHTMLBaseMap()
gm.genMapHTMLMapTop()

#--------------------------------------------------------
# Paste below
#--------------------------------------------------------

gm.genMarker(45.47890524246566, -75.56611403321713,"RCH","RCH","")
gm.genMarker(45.478045775348, -75.56241738538371,"RCH","","")
gm.genMarker(45.479084619032285, -75.5606251644505,"RCH","","")
gm.genMarker(45.479801606521505, -75.55691209024718,"RCH","","")
gm.genMarker(45.478952924359234, -75.55542219985323,"RCH","","")
gm.genMarker(45.4793838133057, -75.55354605510716,"RCH","","")
gm.genMarker(45.479780549057914, -75.5525676092125,"RCH","","")
gm.genMarker(45.470290416814905, -75.55228904428883,"RCH","","")
gm.genMarker(45.47033550770272, -75.55056360470792,"RCH","","")
gm.genMarker(45.46974425824116, -75.55151372730964,"RCH","","")
gm.genMarker(45.468732503626555, -75.55196028059133,"RCH","","")
gm.genMarker(45.4692836321735, -75.55304573662096,"RCH","","")
gm.genMarker(45.46742990378732, -75.55848432094648,"RCH","","")
gm.genMarker(45.476376073783534, -75.55509179544087,"Furnace","Clingan & Buckley's Furnace","Clingan & Buckley's Furnace")
gm.genMarker(45.475871000000005, -75.549315,"Business","Clingan & Buckley","Clingan & Buckley")
gm.genMarker(45.474517021721255, -75.54960034240014,"House","","")
gm.genMarker(45.473646883679656, -75.5498010012587,"House","","")
gm.genMarker(45.47288801824723, -75.5493773809866,"House","","")
gm.genMarker(45.47284327836467, -75.54810303123833,"House","","S.H. Is this a school house?")
gm.genMarker(45.472181522300446, -75.5491934365716,"House","Clingan & Buckley","Clingan & Buckley's house.")
gm.genMarker(45.47128535207631, -75.54777437025605,"House","N. Care","N. Care's house.")
gm.genMarker(45.47358087484639, -75.55173411301125,"House","","")
gm.genMarker(45.47305413188688, -75.55368214361806,"House","","")
gm.genMarker(45.472486551004515, -75.55459703157238,"House","","")
gm.genPolygon('[[45.47062415323124,-75.54889921058958],[45.470599352060646,-75.54886553632893],[45.470599352060646,-75.54886553632893],[45.470599352060646,-75.54886553632893],[45.470550857336605,-75.54886711135852],[45.470550857336605,-75.54886711135852],[45.47018603927984,-75.5488100009852],[45.46626269695714,-75.55528118925226],[45.466430690962596,-75.56275727256723],[45.46776257532926,-75.56412794312976],[45.4680883727609,-75.56173846718401],[45.47139486562246,-75.5545978949154],[45.472010789802596,-75.55064716673857],[45.47062415323124,-75.54889921058958]]','hill')
gm.genPolygon('[[45.4703507858807,-75.54849433378025],[45.470423527956484,-75.54849197075316],[45.47039928059778,-75.54849275842955],[45.47039928059778,-75.54849275842955],[45.47039928059778,-75.54849275842955],[45.47550895013372,-75.54943020529788],[45.47555412288517,-75.54922184149952],[45.47039706479967,-75.54835491190435],[45.4703507858807,-75.54849433378025]]','road')
gm.genPolygon('[[45.47401823117924,-75.54875448978271],[45.475259703353245,-75.54750726977957],[45.47514552762951,-75.54644202051063],[45.47508354573371,-75.54409922108393],[45.474554354612195,-75.54287509358707],[45.47393205280555,-75.54189539876762],[45.470541440982224,-75.54828126203373],[45.47401823117924,-75.54875448978271]]','field')
gm.genPolygon('[[45.47471015181694,-75.56928341100588],[45.47416630161331,-75.56864572101843],[45.473998351089854,-75.567237367277],[45.47334817580819,-75.5660169426807],[45.472330432508876,-75.56456701414244],[45.472449554136695,-75.56290810281226],[45.47193876736061,-75.56130398182765],[45.47223527860265,-75.56012207595242],[45.4720655684579,-75.55861040894452],[45.472511944923546,-75.55769948749278],[45.47290320192891,-75.55637656018523],[45.47384385406091,-75.5545186049693],[45.473774925317436,-75.55324501468522],[45.47402064539375,-75.55192674301661],[45.47431429145947,-75.55057242193489],[45.4740662841773,-75.55023565072113],[45.47384538438927,-75.5515876084161],[45.47364208422111,-75.55252520517138],[45.473441538062694,-75.55363511292354],[45.472723541273226,-75.5557617130891],[45.4724028286119,-75.55694443676632],[45.47191788051957,-75.55847725780251],[45.47186276006945,-75.55958241237649],[45.47169959252789,-75.56151857804278],[45.472162430691185,-75.56315871952734],[45.47213864851625,-75.56471111088939],[45.473082001932326,-75.5660599865205],[45.473737657604836,-75.56762504474739],[45.47395026479396,-75.56879058723617],[45.47444835637883,-75.56960215508987],[45.47471015181694,-75.56928341100588]]','creek')
gm.genPolygon('[[45.4785386654922,-75.56747072259486],[45.482376682661815,-75.55258690818904],[45.48035861839475,-75.55230746930674],[45.47697053415453,-75.55131383236089],[45.476676992788505,-75.55722002965085],[45.47732229067866,-75.55964751928265],[45.476466940562865,-75.56229586542237],[45.4770581434987,-75.5643803069259],[45.478032331902355,-75.56614211537881],[45.4785386654922,-75.56747072259486]]','forest')
gm.genPolygon('[[45.47458021548236,-75.57079908196994],[45.47482248844092,-75.56898519999727],[45.474542549018395,-75.56629708011141],[45.474407276304674,-75.56312269875735],[45.474791456640844,-75.56065400701607],[45.475627401699604,-75.5588931378809],[45.476509503827714,-75.55790135495803],[45.47638519436303,-75.55754413919207],[45.47550348010541,-75.55855999498114],[45.47463405702033,-75.560346027266],[45.4741163318268,-75.56293942420625],[45.474348151203074,-75.56685720213196],[45.47450112800429,-75.56901960237103],[45.474369367326986,-75.57032422180747],[45.47458021548236,-75.57079908196994]]','path')
gm.genPolygon('[[45.468483218018164,-75.56774405268857],[45.468183559464656,-75.56594778870016],[45.468188680328204,-75.5641417291783],[45.46854822985354,-75.56225200755348],[45.469659005000565,-75.55966379871796],[45.471455533611504,-75.5565235516479],[45.47207821239626,-75.55520307462959],[45.4730879947908,-75.55266598886531],[45.47368326430733,-75.55069612768683],[45.47418432104886,-75.55024640616134],[45.47440593041865,-75.54927595729768],[45.47421849920981,-75.5492097997704],[45.47406269290402,-75.55005770359485],[45.4734789002209,-75.55063051568075],[45.472218666156586,-75.55445203579215],[45.4695870406777,-75.55940125022278],[45.468375820738636,-75.5620649355742],[45.46798470785655,-75.56410014018705],[45.468016519938374,-75.56609763311559],[45.468245761978174,-75.567727604881],[45.468483218018164,-75.56774405268857]]','road')
gm.genPolygon('[[45.47883034688282,-75.569578966572],[45.47801589883442,-75.56630576576292],[45.476922627413586,-75.56467924728209],[45.47639395207318,-75.5623362147724],[45.47714335421863,-75.55939806969702],[45.476498341728814,-75.55720332997322],[45.476792571414585,-75.55124546000772],[45.476606300078075,-75.55125150243325],[45.47634978714214,-75.55744895491367],[45.476954390805176,-75.55923559445522],[45.47618844305066,-75.56219834737838],[45.47682447456945,-75.56489914863276],[45.47784770952702,-75.56638342431208],[45.478645986013966,-75.56970530169316],[45.47883034688282,-75.569578966572]]','road')
gm.genPolygon('[[45.480971408160194,-75.55192875745514],[45.48096600212975,-75.55159175323993],[45.47974831525132,-75.5517275830657],[45.47826889853168,-75.55139023288423],[45.47671904196268,-75.55088660910768],[45.47653347388044,-75.54988117198964],[45.475717947965656,-75.5497390725426],[45.47565902740106,-75.54923526432933],[45.47550546361247,-75.54916800556886],[45.47553515574858,-75.54996174408315],[45.47635029484681,-75.5500797766354],[45.47663939507246,-75.55120226412929],[45.47815345181579,-75.55158664187674],[45.47987187119352,-75.55203666496888],[45.480971408160194,-75.55192875745514]]','road')
gm.genPolygon('[[45.47648151295685,-75.54981061188191],[45.47641099018158,-75.54858470476557],[45.47639435176766,-75.54754970694394],[45.476225014377995,-75.54755521133455],[45.47631480279999,-75.54892498105468],[45.476346042956166,-75.54981500986689],[45.47648151295685,-75.54981061188191]]','road')
gm.genPolygon('[[45.47687087541133,-75.55931053879046],[45.47653067037865,-75.55816557593738],[45.47568012606779,-75.55901184438737],[45.47494501349009,-75.56072129679444],[45.47449271273512,-75.56316810846599],[45.47497183326114,-75.56878775129826],[45.47852706676656,-75.56968503665472],[45.47776495529629,-75.56650649994052],[45.4766570528707,-75.56502494967678],[45.47602217600272,-75.56239636410216],[45.47687087541133,-75.55931053879046]]','field')
gm.genPolygon('[[45.4743930228796,-75.56861369266286],[45.47401664763761,-75.563063042353],[45.47448165287219,-75.56035094580591],[45.47544806217013,-75.55827280366786],[45.476247414572306,-75.55740409993686],[45.476520858771714,-75.55120610939605],[45.47623214509365,-75.55010769394426],[45.47555363558575,-75.55005747183162],[45.47540966110456,-75.54953234456801],[45.47445866497999,-75.54939465201525],[45.47430208431021,-75.55019442136445],[45.47392662439163,-75.55319267466088],[45.473918596380344,-75.55480637304412],[45.472754436548975,-75.55722804351875],[45.472252962924195,-75.55871316054665],[45.47227718565676,-75.56022944429667],[45.47207433574502,-75.56131960533357],[45.47257305329714,-75.56284467322631],[45.4724124818749,-75.56446323773861],[45.47406295634979,-75.56703494055887],[45.474237558058206,-75.56842603706117],[45.4743930228796,-75.56861369266286]]','field')
gm.genPolygon('[[45.473931113512165,-75.56939912486759],[45.4736345430451,-75.5677952241939],[45.47305084077381,-75.56624873740435],[45.471957569483024,-75.56462236603782],[45.47201999373094,-75.56322370052234],[45.471570162652704,-75.56157666772874],[45.47187118674654,-75.5581475664083],[45.47340553599589,-75.55345036833722],[45.47385376087005,-75.55076283904216],[45.47365673699941,-75.55115452885425],[45.47231220164233,-75.55500285793222],[45.468635202025574,-75.56239367685403],[45.468325684561144,-75.56423363218238],[45.46830171296834,-75.5659199092822],[45.46863677041439,-75.56781135447338],[45.473931113512165,-75.56939912486759]]','field')
gm.genPolygon('[[45.473983746915984,-75.54936191052397],[45.47085788520694,-75.54883732927352],[45.472123814438405,-75.55065040422355],[45.47149668291384,-75.55486069382462],[45.47162331316825,-75.55536227811622],[45.47342578056253,-75.55048775344868],[45.47394415669924,-75.55006155113523],[45.473983746915984,-75.54936191052397]]','field')
gm.genTestMarker(45.47586813624552, -75.57489324641911)
#--------------------------------------------------------
# Paste above
#--------------------------------------------------------

gm.genHTMLEndMap()
gm.genHTMLEndBody()

gm.close()

# 4. Test generated map using http server on Google Colab

To test the map, we can use an http server on Google Colab. 
* Run the cell below to generate the URL for the test map.
* Start the http server
* Click the URL to launch the map in a web browser

Once your map is ready, the userloc.html file can be placed on an http server that has TLS/https and accessed from a smartphone.

In [10]:
# Start the server, then click on the URL
from google.colab.output import eval_js
print("Click this URL when the server starts.")
print(eval_js("google.colab.kernel.proxyPort(8000)")+"drive/MyDrive/github/sensation/userloc.html")
!python -m http.server 8000

Click this URL when the server starts.
https://ugec3c354v-496ff2e9c6d22116-8000-colab.googleusercontent.com/drive/MyDrive/github/sensation/userloc.html
Serving HTTP on 0.0.0.0 port 8000 (http://0.0.0.0:8000/) ...
127.0.0.1 - - [27/Apr/2021 00:57:59] "GET /drive/MyDrive/github/sensation/userloc.html HTTP/1.1" 200 -
127.0.0.1 - - [27/Apr/2021 00:57:59] "GET /drive/MyDrive/github/sensation/icons/question.png HTTP/1.1" 200 -
127.0.0.1 - - [27/Apr/2021 00:57:59] code 404, message File not found
127.0.0.1 - - [27/Apr/2021 00:57:59] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [27/Apr/2021 00:57:59] "GET /drive/MyDrive/github/sensation/icons/computer.png HTTP/1.1" 200 -
127.0.0.1 - - [27/Apr/2021 00:58:02] code 404, message File not found
127.0.0.1 - - [27/Apr/2021 00:58:02] "GET /drive/MyDrive/github/sensation/icons/travel.png HTTP/1.1" 404 -
127.0.0.1 - - [27/Apr/2021 00:58:05] code 404, message File not found
127.0.0.1 - - [27/Apr/2021 00:58:05] "GET /drive/MyDrive/github/sensation/icon